In [37]:

# Imports

import pandas as pd
import re
import html
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")  # run only once
stop_words = set(stopwords.words("english"))

import sys
sys.path.append("../src")

from text_cleaner import clean_dataframe

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amitianeesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:

QUESTIONS_PATH = "../data/raw/Questions.csv"
ANSWERS_PATH = "../data/raw/Answers.csv"
TAGS_PATH = "../data/raw/Tags.csv"

QUESTIONS_OUTPUT_PATH = "../data/processed/questions_clean.csv"
ANSWERS_OUTPUT_PATH = "../data/processed/answers_clean.csv"

CHUNK_SIZE = 50000  # rows processed at a time(large so chunckwise)


counting questions/answers/tags

In [39]:

question_count = sum(
    len(chunk)
    for chunk in pd.read_csv(QUESTIONS_PATH, encoding="latin1", chunksize=100000)
)

question_count


1264216

In [40]:
answer_count = sum(
    len(chunk)
    for chunk in pd.read_csv(ANSWERS_PATH, encoding="latin1", chunksize=100000)
)

answer_count


2014516

In [41]:
tag_count = sum(
    len(chunk)
    for chunk in pd.read_csv(TAGS_PATH, encoding="latin1", chunksize=100000)
)

tag_count


3750994

In [42]:
dataset_summary = pd.DataFrame({
    "TYpe": ["Questions", "Answers", "Tags"],
    "Count": [question_count, answer_count, tag_count]
})

dataset_summary


,TYpe,Count
0,Questions,1264216
1,Answers,2014516
2,Tags,3750994


In [43]:
sample_df_ques= pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    nrows=5
)

# sample_df_ques.head()
sample_df_ques.head(1)


,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."


In [44]:
sample_df_ques= pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    nrows=5
)

sample_df_ques.shape


(5, 7)

In [45]:
from bs4 import XMLParsedAsHTMLWarning
import warnings

# Suppress XML warning once
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [46]:
tags_df = pd.read_csv("../data/raw/Tags.csv")
print(tags_df.head())

   Id             Tag
0  80            flex
1  80  actionscript-3
2  80             air
3  90             svn
4  90     tortoisesvn


In [47]:
unique_tags = tags_df["Tag"].unique()

print("Total Unique Tags:", len(unique_tags))

Total Unique Tags: 37035


In [48]:
for tag in unique_tags:
    print(tag)

flex
actionscript-3
air
svn
tortoisesvn
branch
branching-and-merging
sql
asp.net
sitemap
algorithm
language-agnostic
colors
color-space
c#
.net
scripting
compiler-construction
c++
oop
class
nested-class
web-services
sql-server
sql-server-2005
deployment
release-management
visual-studio
versioning
windows
registry
installation
database
loops
connection
file-locking
unix
size
msbuild
cruisecontrol.net
web-applications
dns
subdomain
account
.net-3.5
nant
windows-server-2008
sql-server-2008
unit-testing
testing
version-control
postgresql
stored-procedures
triggers
dataset
datatable
asp-classic
vbscript
html
autocomplete
c
architecture
data-structures
flash
video
embed
powershell
cmdlets
optimization
setter
getter
verification
winforms
error-handling
user-controls
paging
php
mysql
security
cracking
hijacked
fogbugz
full-text-search
math
ruby
rake
command-line-interface
64bit
wmi
java
code-coverage
static-analysis
diagram
ado.net
business-logic-layer
objectdatasource
wsdl
jni
java-web-start


In [49]:
def clean_text_pipeline(text):
    if pd.isna(text):
        return ""

    # Decode HTML entities (e.g., &lt; → <)
    text = html.unescape(text)
    try:
        # Remove HTML/XML tags
        text = BeautifulSoup(text, "html.parser").get_text()
    except Exception:
        return ""
    
    # Convert to lowercase
    text = text.lower()

    # Remove non-alphabetic characters
    text = re.sub(r"[^a-z\s]", " ", text)

    # Normalize multiple spaces
    text = re.sub(r"\s+", " ", text)

    # Tokenize
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Join back
    text = " ".join(tokens)

    return text.strip()


In [50]:
unique_tags_df = pd.DataFrame(unique_tags, columns=["Tag"]) 

In [51]:
unique_tags_df.to_csv("../data/processed/unique_tags.csv", index=False)

print("Unique tags file saved successfully.") 

Unique tags file saved successfully.


In [52]:
tag_counts = tags_df["Tag"].value_counts()

print(tag_counts.head(20))


Tag
javascript       124155
java             115212
c#               101186
php               98808
android           90659
jquery            78542
python            64601
html              58976
c++               47591
ios               47009
mysql             42464
css               42308
sql               35782
asp.net           29970
objective-c       26922
ruby-on-rails     25789
.net              24059
c                 23238
iphone            21539
angularjs         20345
Name: count, dtype: int64


In [53]:
# Test Cleaning on Small Subset
test_df = pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    nrows=10000
)

test_df["raw_text"] = (
    test_df["Title"].fillna("") + " " +
    test_df["Body"].fillna("")
)

test_df["clean_text"] = test_df["raw_text"].apply(clean_text_pipeline)

sample = test_df[["raw_text", "clean_text"]].iloc[0]

print("RAW TEXT:\n", sample["raw_text"][:500])
print("\nCLEAN TEXT:\n", sample["clean_text"][:500])


RAW TEXT:
 SQLStatement.execute() - multiple queries in one statement <p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thum

CLEAN TEXT:
 sqlstatement execute multiple queries one statement written database generation script sql want execute adobe air application create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean

In [54]:
questions_df = pd.read_csv(
    "../data/raw/Questions.csv",
    encoding="latin1"
)

questions_df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [55]:
tags = pd.read_csv(
    "../data/processed/unique_tags.csv",
    encoding="latin1"
) 
tags.head()

,Tag
0,flex
1,actionscript-3
2,air
3,svn
4,tortoisesvn


In [56]:
# Load tags
tags_df = pd.read_csv("../data/processed/unique_tags.csv", encoding="latin1")

# Convert to clean lowercase list
tags = (
    tags_df.iloc[:, 0]
    .dropna()
    .astype(str)
    .str.lower()
    .str.strip()
    .tolist()
)

print("Total tags loaded:", len(tags))

Total tags loaded: 37034


In [57]:
# Full Dataset Cleaning (Chunk Processing)
first_chunk = True

for chunk in pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    chunksize=CHUNK_SIZE
):

    # Combine title and body
    chunk["raw_text"] = (
        chunk["Title"].fillna("") + " " +
        chunk["Body"].fillna("")
    )

    # Apply cleaning pipeline
    chunk["clean_text"] = chunk["raw_text"].apply(clean_text_pipeline)

    # Keep only relevant columns
    processed_chunk = chunk[["Id", "Score", "clean_text"]]

    # Write incrementally
    processed_chunk.to_csv(
        QUESTIONS_OUTPUT_PATH,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )

    first_chunk = False

print(" Full preprocessing complete.")


 Full preprocessing complete.


In [58]:
# Create placeholder mapping
tag_placeholder_map = {}

for i, tag in enumerate(tags):
    placeholder = f"TAGTOKEN{i}"
    tag_placeholder_map[tag] = placeholder

# Reverse map (to restore later)
reverse_map = {v: k for k, v in tag_placeholder_map.items()}

In [59]:
def protect_tags(text):
    if pd.isna(text):
        return ""
    
    text = text.lower()
    
    for tag, placeholder in tag_placeholder_map.items():
        # replace whole word matches only
        text = re.sub(rf"\b{re.escape(tag)}\b", placeholder, text)
    
    return text
    

In [60]:
# from bs4 import BeautifulSoup

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ")

In [61]:
import os

os.makedirs(os.path.dirname(QUESTIONS_OUTPUT_PATH), exist_ok=True)


In [62]:
#Normal Cleaning (Without Damaging Tags)
def clean_text(text):
    text = remove_html(text)
    
    # remove special characters but keep placeholders
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    
    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [63]:
#Restore Tags Back
def restore_tags(text):
    for placeholder, tag in reverse_map.items():
        text = text.replace(placeholder, tag)
    return text

In [64]:
# verify
clean_df_sample = pd.read_csv(QUESTIONS_OUTPUT_PATH, nrows=5)

clean_df_sample.head()


,Id,Score,clean_text
0,80,26,sqlstatement execute multiple queries one stat...
1,90,144,good branching merging tutorials tortoisesvn r...
2,120,21,asp net site maps anyone got experience creati...
3,180,53,function creating color wheels something pseud...
4,260,49,adding scripting functionality net application...


In [65]:
def full_clean_pipeline(text):
    #Final Pipeline Function
    text = protect_tags(text)
    text = clean_text(text)
    text = restore_tags(text)
    return text

In [66]:
original_count = sum(
    len(chunk)
    for chunk in pd.read_csv(QUESTIONS_PATH, encoding="latin1", chunksize=100000)
)

clean_count = sum(
    len(chunk)
    for chunk in pd.read_csv(QUESTIONS_OUTPUT_PATH, chunksize=100000)
)

print("Original rows:", original_count)
print("Cleaned rows:", clean_count)


Original rows: 1264216
Cleaned rows: 1264216


In [ ]:
#Apply to DataFrame
questions_df["cleaned_body"] = questions_df["Body"].apply(full_clean_pipeline)

answers cleaning

In [ ]:
sample_df_ans= pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    nrows=5
)

# sample_df_ans.head()
sample_df_ques.head()


In [ ]:
# # Test Cleaning on Small Subset of ans
# test_df_ans = pd.read_csv(
#     ANSWERS_PATH,
#     encoding="latin1",
#     nrows=10000
# )

# test_df_ans["raw_text"] = (
#     test_df_ans["Body"].fillna("")
# )

# test_df_ans["clean_text"] = test_df_ans["raw_text"].apply(clean_text_pipeline)

# sample = test_df_ans[["raw_text", "clean_text"]].iloc[0]

# print("RAW TEXT:\n", sample["raw_text"][:500])
# print("\nCLEAN TEXT:\n", sample["clean_text"][:500])


In [ ]:
# Full Dataset Cleaning (Chunk Processing)
first_chunk = True

for chunk in pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    chunksize=CHUNK_SIZE
):

    # Combine title and body
    chunk["raw_text"] = (
        chunk["Body"].fillna("")
    )

    # Apply cleaning pipeline
    chunk["clean_text"] = chunk["raw_text"].apply(clean_text_pipeline)

    # Keep only relevant columns
    processed_chunk = chunk[["Id","ParentId", "Score", "clean_text"]]

    # Write incrementally
    processed_chunk.to_csv(
        ANSWERS_OUTPUT_PATH,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )

    first_chunk = False

print(" Full preprocessing complete.")


In [ ]:
import pandas as pd
answers_sample = pd.read_csv("../data/processed/answers_clean.csv", nrows=5)
answers_sample.head()

In [ ]:
# TF-IDF VECTORIZATION

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

# Load cleaned questions
clean_questions = pd.read_csv("../data/processed/questions_clean.csv")

vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=5,
    max_df=0.8,
    ngram_range=(1,2),
    sublinear_tf=True
)

tfidf_questions = vectorizer.fit_transform(clean_questions["clean_text"].fillna(""))

import joblib
joblib.dump(vectorizer, "../data/processed/tfidf_vectorizer.pkl")

print("Questions TF-IDF Shape:", tfidf_questions.shape)

# Save sparse matrix
save_npz("../data/processed/questions_tfidf.npz", tfidf_questions)

print("Questions TF-IDF saved successfully.")

In [ ]:
import pandas as pd
# Load the newly cleaned answers (now has ParentId)
answers_clean = pd.read_csv("../data/processed/answers_clean.csv")
print("Answers shape:", answers_clean.shape)
print(answers_clean.head(3))

In [ ]:
# Group all answers by the question they belong to (ParentId = Question Id)
answer_stats = answers_clean.groupby("ParentId").agg(
    avg_answer_score=("Score", "mean"),
    answer_count=("Score", "count"),
    max_answer_score=("Score", "max")
).reset_index()
# Rename ParentId to question_id for clarity
answer_stats.rename(columns={"ParentId": "question_id"}, inplace=True)
print("Answer stats shape:", answer_stats.shape)
answer_stats.head(5)

In [ ]:
# STEP 3: Bayesian Smoothing + Normalize to 0-1
# (Replaces the simple min-max normalization)

global_mean = answer_stats["avg_answer_score"].mean()
C = answer_stats["answer_count"].mean()  # confidence factor

print(f"Global mean score: {global_mean:.2f}")
print(f"Confidence factor C: {C:.2f}")

# Apply Bayesian smoothing
answer_stats["bayesian_avg_score"] = (
    (C * global_mean + answer_stats["avg_answer_score"] * answer_stats["answer_count"])
    / (C + answer_stats["answer_count"])
)

# Normalize to 0-1
min_b = answer_stats["bayesian_avg_score"].min()
max_b = answer_stats["bayesian_avg_score"].max()

answer_stats["avg_score_normalized"] = (
    (answer_stats["bayesian_avg_score"] - min_b) / (max_b - min_b)
)

print("\nNormalized score range:")
print("Min:", answer_stats["avg_score_normalized"].min())
print("Max:", answer_stats["avg_score_normalized"].max())
print("Mean:", answer_stats["avg_score_normalized"].mean().round(4))

answer_stats.head(5)


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)

answer_stats["cluster"] = kmeans.fit_predict(
    answer_stats[['avg_score_normalized']]
)
cluster_order = (
    answer_stats.groupby('cluster')['avg_score_normalized']
    .mean()
    .sort_values()
    .index
)

mapping = {
    cluster_order[0]: "Hard",
    cluster_order[1]: "Medium",
    cluster_order[2]: "Easy"
}

answer_stats["difficulty"] = (
    answer_stats["cluster"].map(mapping)
)

answer_stats.head()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

plt.scatter(
    answer_stats.index,
    answer_stats['avg_score_normalized'],
    c=answer_stats['cluster'],
    cmap='viridis',
    s=60
)

plt.xlabel("Question Index")
plt.ylabel("Normalized Score (0–1)")
plt.title("K-Means Clustering of Question Difficulty")

plt.show()

In [ ]:
from scipy.sparse import load_npz

# Load cleaned answers
clean_answers = pd.read_csv("../data/processed/answers_clean.csv")

# Load saved vectorizer
vectorizer = joblib.load("../data/processed/tfidf_vectorizer.pkl")

# Transform answers (DO NOT FIT AGAIN)
tfidf_answers = vectorizer.transform(clean_answers["clean_text"].fillna(""))

print("Answers TF-IDF Shape:", tfidf_answers.shape)

# Save sparse matrix
save_npz("../data/processed/answers_tfidf.npz", tfidf_answers)

print("Answers TF-IDF saved successfully.")


In [ ]:
# Scalling the score and made it till out of 100


# calculated the averge by group of that that particular question id.
# clean_answers['average_score'] = (clean_answers.groupby('Id')['score_out_of_100'].transform('mean'))

# # added a new column of difficulty label and hardcoded the bins such that if average score defines the difficulty.
# def assign_difficulty(score):
#     if score >= 70:
#         return "Easy"
#     elif score >= 40:
#         return "Medium" 
#     else:
#         return "Hard"

# clean_answers['difficulty'] = (
#     clean_answers['average_score']
#     .apply(assign_difficulty)
# )

clean_answers.head()

In [ ]:
# Load cleaned questions
questions_clean = pd.read_csv("../data/processed/questions_clean.csv")

# Merge difficulty labels
questions_with_labels = questions_clean.merge(
    answer_stats[["question_id", "difficulty"]],
    left_on="Id",
    right_on="question_id",
    how="inner"
)

print("After merge shape:", questions_with_labels.shape)
questions_with_labels.head()

In [ ]:
# SAMPLING 
questions_sampled = questions_with_labels.groupby("difficulty", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50000), random_state=42)
).reset_index(drop=True)

print("Sampled shape:", questions_sampled.shape)
print(questions_sampled["difficulty"].value_counts())


In [ ]:
X_text = questions_sampled["clean_text"].fillna("")
y = questions_sampled["difficulty"]


In [ ]:
from sklearn.model_selection import train_test_split

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=5,
    max_df=0.8,
    ngram_range=(1,1),   # changed
    sublinear_tf=True
)

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="saga",      # good for sparse data
    multi_class="multinomial",
    random_state=42
)

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

In [ ]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()

for i, class_label in enumerate(model.classes_):
    top10 = np.argsort(model.coef_[i])[-10:]
    print(f"\nTop words for {class_label}:")
    print([feature_names[j] for j in top10])

In [ ]:
# TEST ON SAMPLE QUESTIONS — see predictions in action
import pandas as pd

# Pick 10 random questions from test set
sample_indices = X_test_text.sample(10, random_state=42).index
sample_texts = X_test_text.loc[sample_indices]
sample_actual = y_test.loc[sample_indices]

# Transform and predict
sample_tfidf = vectorizer.transform(sample_texts)
sample_preds = model.predict(sample_tfidf)
sample_probs = model.predict_proba(sample_tfidf)

# Display results
results = pd.DataFrame({
    "question_snippet": sample_texts.str[:80].values,
    "actual_difficulty": sample_actual.values,
    "predicted_difficulty": sample_preds,
    "confidence": sample_probs.max(axis=1).round(3)
})

print("Model Predictions on Sample Questions:\n")
print(results.to_string(index=False))

# Count correct predictions
correct = (results["actual_difficulty"] == results["predicted_difficulty"]).sum()
print(f"\nCorrect: {correct}/10")


NameError: name 'X_test_text' is not defined